In [ ]:
import numpy as np

import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_rows', 500)

In [ ]:
# file pathes to input data
fp_ut = 'UT.csv'     
fp_buh =  'BUH.csv'

In [ ]:
# ABOUT THE DATA
# both files has similar columns but the UT has additional columns 'Серия', "КодСерии"
# both files has 'quantity' and 'sum' fields

In [ ]:
df_ut = pd.read_csv(fp_ut, sep=';', dtype={'КодСерии': 'str','КодНоменклатуры':'str','Количество':float})

In [ ]:
df_ut['ЭтоБухгалтерия'] = False # to distinguish ut.csv rows from buh.csv

In [ ]:
df_buh = pd.read_csv(fp_buh,sep=';', dtype={'КодНоменклатуры': 'str','Сумма':float, 'Количество':float}) # 'Сумма':'str' is intentionally left

In [ ]:
df_buh.Сумма.dtype

In [ ]:
df_buh['ЭтоБухгалтерия'] = True # ADDING FLAG TO MARK THE RECORDS AS BUH

In [ ]:
DF_UT_rows = df_ut.shape[0]
print(df_ut.shape)
print(df_ut.sum(numeric_only=True)) # output SUMM
df_ut = df_ut.sort_values(by=['КодНоменклатуры'])

In [ ]:
DF_BUH_rows = df_buh.shape[0]
print(df_buh.shape) # 
print(df_buh.sum(numeric_only=True)) # output SUMM
df_buh = df_buh.sort_values(by=['КодНоменклатуры'])

In [ ]:
# CONCATENTATION OF buh and ut
df = pd.concat([df_ut, df_buh], sort=True)
df.reset_index(drop=True)

In [ ]:
df['КодНоменклатуры'] = df['КодНоменклатуры'].astype(str) # convert to string
df['КодНоменклатуры'] = df['КодНоменклатуры'].str.strip() # removing whitespace
df['КодНоменклатуры'] = df['КодНоменклатуры'].replace('nan','')

In [ ]:
df['Количество'] = df['Количество'].fillna(0.0)
df['Количество'] = df['Количество'].astype(float)

In [ ]:
df.to_excel('UT_BUH.xlsx',index_label=False,index=False)

In [ ]:
print("United DF, rows:", df.shape[0]," cols:",df.shape[1])

In [ ]:
subset_of_rows=['КодНоменклатуры','Номенклатура','Склад']
duplicated_df_series = df.duplicated(subset=subset_of_rows,keep=False)

In [ ]:
# Select all duplicate rows based on all columns
duplicateRowsDF = df[duplicated_df_series]
duplicateRowsDF = duplicateRowsDF.reset_index(drop=True)

In [ ]:
print("Duplicates DF, rows:",duplicateRowsDF.shape[0]," cols:",duplicateRowsDF.shape[1])

In [ ]:
df.sum(numeric_only=True)

In [ ]:
DF_rows_before_reducing = df.shape[0]
print("DF before duplicates removing, rows:",df.shape[0]," cols:",df.shape[1])
DF_SUM_BEFORE = df.sum(numeric_only=True)
print(DF_SUM_BEFORE["Сумма"])

In [ ]:
# Removing intersection of 2 UT and BUH by their intersection
df.drop_duplicates(subset=subset_of_rows, 
                     keep = False, inplace = True) 
print(df.shape)
DF_rows_AFTER_reducing = df.shape[0]
DF_SUM_AFTER = df.sum(numeric_only=True)

In [ ]:
##df

In [ ]:
duplicateRowsDF_rows_number = duplicateRowsDF.shape[0]
duplicateRowsDF_summ = duplicateRowsDF.sum(numeric_only=True)

In [ ]:
print("DF_UT_rows:",DF_UT_rows)
print("DF_BUH_row:",DF_BUH_rows)
print()
print("DF_rows_before_reducing:",DF_rows_before_reducing)
print("DF_rows_AFTER_reducing:",DF_rows_AFTER_reducing)
print()
print("duplicateRowsDF_rows_number:",duplicateRowsDF_rows_number)

In [ ]:
print("*******************")
print("Сумма до удаления дублей:",DF_SUM_BEFORE["Сумма"]) 
print("Сумма после удаления дублей:",DF_SUM_AFTER["Сумма"])
print()
print("Сумма дубликатов:            ", duplicateRowsDF_summ["Сумма"])
print('разница до от разницы после', round(DF_SUM_BEFORE["Сумма"]-DF_SUM_AFTER["Сумма"], 2)) # если тут разнциа то это ошибка

In [ ]:
print("rows dupl",duplicateRowsDF.shape[0])
print(duplicateRowsDF.sum(numeric_only=True))

In [ ]:
# As far as the the buh.csv has no "СерияНоменклатуры" field consider the rows of subset from buh as unique
rows_of_buh = duplicateRowsDF[duplicateRowsDF['ЭтоБухгалтерия'] == True]

In [ ]:
print(rows_of_buh.shape)
print(rows_of_buh.sum(numeric_only=True))

In [ ]:
counter = 0
size_of_rows = rows_of_buh.shape[0]
for index, row in rows_of_buh.iterrows():   
    ##print("TEMP SERIES BEGIN")
    ##print("***************************************")
    temp_df  = duplicateRowsDF.query('КодНоменклатуры == \''+row['КодНоменклатуры']+'\' and Номенклатура ==\''+row['Номенклатура']+'\' and Склад == \''+row['Склад']+'\'', inplace = False)
    buh_sum = row['Сумма']
    counter+=1
    print("counter:",counter,"/",size_of_rows)
    ##print("бухгалтерская сумма:", buh_sum)  
        
    # Removing the buh string from temp_df otherwirse I won't be able to count the quantity of series in UT
    ##############################
    # Get names of indices for which ЭтоБухгалтерия is true
    indexNames = temp_df[temp_df['ЭтоБухгалтерия'] == True ].index
    # Delete these row indices from dataFrame
    temp_df.drop(indexNames , inplace=True)
    ##############################
    
    temp_number_of_series = temp_df['Количество'].sum() 
    ##print("Количество по сериям: ",temp_number_of_series) 
    
    # define the price
    if temp_number_of_series != 0:
        price = buh_sum / temp_number_of_series
        price = round(price,2)
    else:
        price = 0.0
        
    ##print("Price of series is:", price)
    
    summ_accum = 0.0 # the_sum_remains check 
    
    # Summ distribution to series 
    for temp_index,temp_row in temp_df.iterrows():
        current_sum = price * temp_row['Количество']
        duplicateRowsDF.at[temp_index,'Сумма'] = round(current_sum,2)
        ##print("Teh summ is", duplicateRowsDF.at[temp_index,'Сумма'])
        the_last_index = temp_index  # hope it won't clear iself after the loop
        summ_accum+=round(current_sum,2)
        
    # the_sum_remains check
    if summ_accum != buh_sum:
        print("difference in ",row['КодНоменклатуры'],row['Номенклатура'],row['Склад'], " sum if difference between ",buh_sum," and ",summ_accum)
        summ_dif = buh_sum - summ_accum
        duplicateRowsDF.at[the_last_index, 'Сумма'] += round(summ_dif,2)  # add the sum to the last row with the series
  
    ##display(temp_df)

In [ ]:
# Removing buh-string from duplicates dataframe - we've already got the sum to distibute among set of Series rows
###############################################################################################
# Get names of indexes for which ЭтоБухгалтерия is true
indexNames = duplicateRowsDF[duplicateRowsDF['ЭтоБухгалтерия'] == True ].index
# Delete these row indexes from dataFrame
duplicateRowsDF.drop(indexNames , inplace=True)
##############################

In [ ]:
# adding fixed duplicates to the df
df = pd.concat([df, duplicateRowsDF], sort=True) # sets union
df.reset_index(drop=True)


In [ ]:
df.shape

In [ ]:
df.sum(numeric_only=True)["Количество"]

In [ ]:
round(df.sum(numeric_only=True)["Сумма"],2)

In [ ]:
df = df.sort_values(by=['КодНоменклатуры'])

In [ ]:
##df

In [ ]:
# THE FINAL RESULT BEING MADE HERE
###############################################################################################
df.to_excel('final_UT_BUH.xlsx',index_label=False,index=False,float_format='%.2f')

In [ ]:
# All code needed to check the difference of concatenating ut and buh listed below
###############################################################################################
# reread the new file
fp_changed ="final_UT_BUH_changeable.xlsx"
#df_changed = pd.read_csv(fp_changed, sep=';', encoding='cp1251', dtype={'КодСерии': 'str','КодНоменклатуры':'str','Количество':float})
df_changed = pd.read_excel(fp_changed)

In [ ]:
df_changed.dtypes

In [ ]:
print(df_changed.shape[0])
print()
print(df_ut.shape[0])

In [ ]:
df_changed = df_changed.drop(['ЭтоБухгалтерия'],  axis = 1)

In [ ]:
df_ut = df_ut.drop(['ЭтоБухгалтерия'],  axis = 1)

In [ ]:
df_changed = df_changed.drop(['Сумма'],  axis = 1)
df_ut = df_ut.drop(['Сумма'],  axis = 1)

In [ ]:
# ATTENTION!
# Code below check the data in memory from df variable! Not from xls file
############################################################################################

df_mem = df.copy(deep=True)  # deep=False - means no indices
# dropping unwanted columns
df_mem = df_mem.drop(['ЭтоБухгалтерия'],  axis = 1)
df_mem = df_mem.drop(['Сумма'],  axis = 1)
# converting columns
df_mem['КодНоменклатуры'] = df_mem['КодНоменклатуры'].str.strip()
df_mem['КодСерии'] = df_mem['КодСерии'].str.strip()
df_mem['Номенклатура'] = df_mem['Номенклатура'].str.strip()
df_mem['Серия'] = df_mem['Серия'].str.strip()
df_mem['Склад'] = df_mem['Склад'].str.strip()
# show example
display(df_mem.head())
print(df_mem.dtypes)
df_mem.reset_index(inplace=True,drop=True)
#display(df_mem.head)
print(df_mem.columns.tolist())
print(df_mem.dtypes)
print(df_mem.shape)

In [ ]:
df_mem.head()

In [ ]:
df_ut['КодНоменклатуры'] = df_ut['КодНоменклатуры'].str.strip()
df_ut['КодСерии'] = df_ut['КодСерии'].str.strip()
df_ut['Номенклатура'] = df_ut['Номенклатура'].str.strip()
df_ut['Серия'] = df_ut['Серия'].str.strip()
df_ut['Склад'] = df_ut['Склад'].str.strip()

print(df_ut.columns.tolist())

print(df_ut.dtypes)

df_ut = df_ut.reindex(columns=df_mem.columns.tolist())
print(df_ut.reset_index(inplace=True,drop=True))
print(df_ut.shape)

In [ ]:
df_ut.head()

In [ ]:
# Check columns names for equality
################################
cols1 = df_mem.columns.tolist()
cols2 = df_ut.columns.tolist()
print(cols1)
print(cols2)
if cols1 == cols2:
    print('column names are equal')

In [ ]:
# Code below used for column content collisions
#############################################################
# eg I found the rows with same code but different names - 
# it means the same item in UT and Buh has the different names but the same code

# specify the column to search for duplicates
#COL_NAME = 'КодНоменклатуры'
COL_NAME = 'Номенклатура'

# Prepare the first set 
df_mem_items = list(set(df_mem[COL_NAME].tolist())) # Column folding - get All unique codes of items
print(len(df_mem_items))
df_mem_items.sort()
print(df_mem_items[:10]) # show 10 

# Prepare the second set 
ut_mem_items = list(set(df_ut[COL_NAME].tolist())) # column folding
print(len(ut_mem_items))
ut_mem_items.sort()
print(ut_mem_items[:10])

 
# Function to get the intersection of two lists using set() method 
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

common_values = intersection(df_mem_items, ut_mem_items) # get the intersection

print(len(common_values)) # how many common items in common items set
print(common_values[:10])

print("len of df_mem_items ", len(df_mem_items)) # how many common items in the first set
print()

# MAKING LIST OF UNIQUE CODES FROM df_mem
##############################################
df_mem_count_of_unique_values = 0
df_mem_unique_values = []
# unique df_mem_items
for code in df_mem_items:
    if code not in common_values:
        df_mem_unique_values.append(code)
        df_mem_count_of_unique_values += 1    
print("Number of unique values in df_mem_items", df_mem_count_of_unique_values)
print()
print(df_mem_unique_values)

# MAKING LIST OF UNIQUE CODES FROM df_mem
##############################################
df_ut_count_of_unique_values = 0
df_ut_unique_values = []
# unique df_mem_items
for code in ut_mem_items:
    if code not in common_values:
        df_ut_unique_values.append(code)
        df_ut_count_of_unique_values += 1
        
print("Number of unique values in df_ut ", df_ut_count_of_unique_values)
print()
print(df_ut_unique_values)

In [ ]:
df_names_are_different_in_UT_BUH = df_mem[df_mem[COL_NAME].isin(df_mem_unique_values)]

In [ ]:
df_ut[df_ut["КодНоменклатуры"].isin(df_mem_unique_values)] # empty is right

In [ ]:
print(df_mem.shape)
print(df_ut.shape)
common = df_mem.merge(df_ut,on=['КодНоменклатуры','КодСерии','Номенклатура','Серия','Склад','Количество'])
print(common.shape) #

In [ ]:
# out the dif in excel
df_names_are_different_in_UT_BUH.to_excel('different_names.xlsx',index_label=False,index=False,float_format='%.2f') 